In [1]:
from plot import *
from fft import *
from maths import rmse, mae, linear_interpolate, cosine_interpolate, cubic_interpolate
from maths import catmull_rom_interpolate, hermite_interpolate, fit_kochanek_bartels_spline


In [18]:
def dft2_at_time_and_level(data: np.ndarray, quantile: float = 0.75):
    fft = np.fft.rfft2(data)
    amplitudes = np.abs(fft)

    fft_real = []
    fft_imag = []
    fft_i_indices = []
    fft_j_indices = []

    cutoff_amp = np.quantile(amplitudes, quantile)

    for j in range(289):
        for i in range(361):
            if amplitudes[i, j] < cutoff_amp:
                continue

            fft_real.append(fft[i, j].real)
            fft_imag.append(fft[i, j].imag)
            fft_i_indices.append(i)
            fft_j_indices.append(j)

    fft_real = (np.array(fft_real, dtype="float32") / 512).astype("float16")
    fft_real = encode_zlib(fft_real)

    fft_imag = (np.array(fft_imag, dtype="float32") / 512).astype("float16")
    fft_imag = encode_zlib(fft_imag)

    fft_i_indices = np.array(fft_i_indices, dtype="int16")
    fft_i_indices = encode_difference_uint8(fft_i_indices)
    fft_i_indices = encode_zlib(fft_i_indices)

    fft_j_indices = np.array(fft_j_indices, dtype="int16")
    fft_j_indices = encode_difference_uint8(fft_j_indices)
    fft_j_indices = encode_zlib(fft_j_indices, strategy=0)

    return fft_real, fft_imag, fft_i_indices, fft_j_indices


def idft2_at_time_and_level(fft_real, fft_imag, fft_i_indices, fft_j_indices):
    ifft = np.zeros((361, 289), dtype="complex64")

    fft_real = decode_zlib(fft_real, dtype="float16")
    fft_imag = decode_zlib(fft_imag, dtype="float16")
    fft = fft_real.astype("complex64") * 512 + fft_imag.astype("complex64") * 512j

    fft_i_indices = decode_zlib(fft_i_indices)
    fft_i_indices = decode_difference_uint8(fft_i_indices)

    fft_j_indices = decode_zlib(fft_j_indices)
    fft_j_indices = decode_difference_uint8(fft_j_indices)

    for idx in range(len(fft)):
        ifft[fft_i_indices[idx], fft_j_indices[idx]] = fft[idx]

    return np.fft.irfft2(ifft)


def plot_dft2_at_time_and_level(filename: str, variable: str, time: int, level: int, **kwargs):
    data = load_variable_at_time_and_level(filename, variable, time, level)

    fft = dft2_at_time_and_level(data, **kwargs)
    prediction = idft2_at_time_and_level(*fft)

    title = f"{format_variable(variable)} at {format_level(level)}" \
            f" on {format_date(filename)} at {format_time(time, filename)}"
    output = f"{variable}/{kwargs['quantile']}/{format_level(level, for_output=True)}" \
             f"-{format_date(filename, for_output=True)}-{format_time(time, filename)}"

    fig, ax1, ax2 = create_1x2_plot(title, sharey=True, sharex=True)
    ax1.imshow(data, cmap=cmr.arctic, origin="lower", extent=[-180, 180, -90, 90], aspect="auto")
    ax2.imshow(prediction, cmap=cmr.arctic, origin="lower", extent=[-180, 180, -90, 90], aspect="auto")

    ax1.xaxis.set_major_formatter(FormatStrFormatter("%d°"))
    ax1.yaxis.set_major_formatter(FormatStrFormatter("%d°"))

    fig.suptitle(title, fontsize=8, y=0.96)

    plt.savefig("assets/2D-dft-graphs/" + output + ".png", dpi=300)
    plt.show()

    title = f"Error ({get_units_from_variable(variable)}) at {format_level(level)} " \
            f" on {format_date(filename)} at {format_time(time, filename)}"

    fig, ax1, ax2 = create_1x2_plot(title, sharey=True, sharex=True)
    ax1.imshow(np.abs(prediction - data), cmap="hot", origin="lower", extent=[-180, 180, -90, 90], aspect="auto")
    ax2.imshow((prediction - data) ** 2, cmap="hot", origin="lower", extent=[-180, 180, -90, 90], aspect="auto")

    ax1.xaxis.set_major_formatter(FormatStrFormatter("%d°"))
    ax1.yaxis.set_major_formatter(FormatStrFormatter("%d°"))

    fig.suptitle(title, fontsize=8, y=0.96)
    ax1.set_title("Absolute Error (m/s)", fontsize=8)
    ax2.set_title("Squared Error (m²/s²)", fontsize=8)

    output += "-error"
    plt.savefig("assets/2D-dft-graphs/" + output + ".png", dpi=300)
    plt.show()

    lines = f"""
    Original Stdev: {data.astype('float32').std()} m/s
    Predicted MAE:  {mae(data, prediction)} m/s
    Predicted RMSE: {rmse(data, prediction)} m/s

    Frequencies: {len(fft[0])}
    Size/time: {sum(el.nbytes for el in fft) * 36 / (1000 ** 2)} MB
    Size/day: {sum(el.nbytes for el in fft) * 36 * 8 / (1000 ** 2)} MB
    Size/year: {sum(el.nbytes for el in fft) * 36 * 8 * 365 / (1000 ** 2)} MB
    """
    print(lines)
    with open("assets/2D-dft-graphs/" + output + ".txt", "w") as file:
        file.writelines(lines)


In [ ]:
%matplotlib notebook

for var in ["U", "V"]:
    plot_dft2_at_time_and_level("MERRA2.tavg3_3d_asm_Nv.YAVG0101.nc4", var,
                                time=0, level=35, quantile=0.9935)


In [44]:
def fit_dft2_at_time(filename: str, variable: str, time: int, verbose: bool = True, **kwargs):
    data = load_variable_at_time(filename, variable, time)
    prediction = np.zeros((36, 361, 576), dtype="float32")

    dfts = []
    for lev in (tqdm(range(36)) if verbose else range(36)):
        fft = dft2_at_time_and_level(data[lev], **kwargs)
        dfts.append(fft)
        prediction[lev] = idft2_at_time_and_level(*fft)

    data = data.astype("float32")
    if verbose:
        size = sum(el.nbytes for dft in dfts for el in dft)
        lines = f"""
        Original Stdev: {data.std()} m/s
        Predicted MAE:  {mae(data, prediction)} m/s
        Predicted RMSE: {rmse(data, prediction)} m/s

        Frequencies: {len(fft[0])}
        Size/time: {size / (1000 ** 2)} MB
        Size/day: {size * 8 / (1000 ** 2)} MB
        Size/year: {size * 8 * 365 / (1000 ** 2)} MB
        """
        print(lines)
    else:
        return data, prediction, dfts


def fit_dft2_on_day(filename: str, variable: str, **kwargs):
    data_variance = 0
    mae_error = np.zeros((36, 361, 576), dtype="float32")
    mse_error = np.zeros((36, 361, 576), dtype="float32")
    all_dfts = []

    for time in tqdm(range(8)):
        data, pred, dfts = fit_dft2_at_time(filename, variable, time, verbose=False, **kwargs)
        all_dfts.append(dfts)

        data_variance += data.var()
        mae_error += abs(data - pred)
        mse_error += (data - pred) ** 2

    size = sum(el.nbytes for dfts in all_dfts for dft in dfts for el in dft)
    lines = f"""
    Original Stdev: {(data_variance / 8) ** 0.5} m/s
    Predicted MAE:  {(mae_error / 8).mean()} m/s
    Predicted RMSE: {(mse_error / 8).mean() ** 0.5} m/s

    Size/day: {size / (1000 ** 2)} MB
    Size/year: {size * 365 / (1000 ** 2)} MB
    """
    print(lines)


def fit_interpolated_dft2_at_time(filename: str, variable: str, time: int, skip_levels: int = 2,
                                  interpolation: callable = linear_interpolate, verbose: bool = True,
                                  **kwargs):
    data = load_variable_at_time(filename, variable, time)
    prediction = np.zeros((36, 361, 576), dtype="float32")

    preds = []
    dfts = []
    levels = sorted(set(range(0, 36, skip_levels)).union({35}))
    for lev in (tqdm(levels) if verbose else levels):
        fft = dft2_at_time_and_level(data[lev], **kwargs)
        dfts.append(fft)
        preds.append(idft2_at_time_and_level(*fft))

    for lev in range(36):
        if lev in levels:
            prediction[lev] = preds[levels.index(lev)]
            continue

        pred_idx = levels.index(lev - lev % skip_levels)
        t = (lev - levels[pred_idx]) / (levels[pred_idx + 1] - levels[pred_idx])
        prediction[lev] = interpolation(preds, pred_idx, t)

    data = data.astype("float32")
    if verbose:
        size = sum(el.nbytes for dft in dfts for el in dft)
        print(f"Original Stdev: {data.std()} m/s")
        print(f"Predicted RMSE: {rmse(data, prediction)} m/s")
        print(f"Predicted MAE:  {mae(data, prediction)} m/s")
        print(f"Size/time: {size / (1000 ** 2)} MB")
        print(f"Size/day: {size * 8 / (1000 ** 2)} MB")
        print(f"Size/year: {size * 8 * 365 / (1000 ** 2)} MB")
    else:
        return data, prediction, dfts


def fit_interpolated_dft2_on_day(filename: str, variable: str, quantile: float, skip_levels: int,
                                 interpolation: callable = linear_interpolate):
    data_variance = 0
    mae_error = np.zeros((36, 361, 576), dtype="float32")
    mse_error = np.zeros((36, 361, 576), dtype="float32")
    all_dfts = []

    for time in tqdm(range(8)):
        data, pred, dfts = fit_interpolated_dft2_at_time(filename, variable, time, skip_levels, interpolation,
                                                         quantile=quantile, verbose=False)
        all_dfts.append(dfts)

        data_variance += data.var()
        mae_error += abs(data - pred)
        mse_error += (data - pred) ** 2

    size = sum(el.nbytes for dfts in all_dfts for dft in dfts for el in dft)
    lines = f"""
    Original Stdev: {(data_variance / 8) ** 0.5} m/s
    Predicted MAE:  {(mae_error / 8).mean()} m/s
    Predicted RMSE: {(mse_error / 8).mean() ** 0.5} m/s

    Size/day: {size / (1000 ** 2)} MB
    Size/year: {size * 365 / (1000 ** 2)} MB
    """
    print(lines)


def fit_dft2_hermite_interpolated_at_time(filename: str, variable: str, time: int, quantile: float, skip_levels: int,
                                          verbose: bool = True):
    data = load_variable_at_time(filename, variable, time)
    prediction = np.zeros((36, 361, 576), dtype="float32")

    preds = []
    dfts = []
    levels = sorted(set(range(0, 36, skip_levels)).union({35}))
    for lev in (tqdm(levels) if verbose else levels):
        q = quantile[lev // skip_levels] if isinstance(quantile, list) else quantile

        fft = dft2_at_time_and_level(data[lev], quantile=q)
        dfts.append(fft)
        preds.append(idft2_at_time_and_level(*fft))

    tensions = []
    biases = []

    for lev in range(36):
        if lev % skip_levels == 0:
            continue

        pred_idx = levels.index(lev - lev % skip_levels)
        t = (lev - levels[pred_idx]) / (levels[pred_idx + 1] - levels[pred_idx])

        best_tension, best_bias = fit_kochanek_bartels_spline(data[lev], preds, pred_idx, t)
        tensions.append(best_tension)
        biases.append(best_bias)

    tensions = np.array(tensions, dtype="float16")
    biases = np.array(tensions, dtype="float16")

    for lev in range(36):
        if lev in levels:
            prediction[lev] = preds[levels.index(lev)]
            continue

        pred_idx = levels.index(lev - lev % skip_levels)
        t = (lev - levels[pred_idx]) / (levels[pred_idx + 1] - levels[pred_idx])
        prediction[lev] = hermite_interpolate(preds, pred_idx, t, tensions[lev - pred_idx - 1], 0)

    data = data.astype("float32")
    if verbose:
        size = sum(el.nbytes for dft in dfts for el in dft)
        print(f"Original Stdev: {data.std()} m/s")
        print(f"Predicted RMSE: {rmse(data, prediction)} m/s")
        print(f"Predicted MAE:  {mae(data, prediction)} m/s")
        print(f"Size/time: {size / (1000 ** 2)} MB")
        print(f"Size/day: {size * 8 / (1000 ** 2)} MB")
        print(f"Size/year: {size * 8 * 365 / (1000 ** 2)} MB")
    else:
        return data, prediction, dfts, tensions, biases


def fit_dft2_hermite_interpolated_on_day(*args, **kwargs):
    data_variance = 0
    mae_error = np.zeros((36, 361, 576), dtype="float32")
    mse_error = np.zeros((36, 361, 576), dtype="float32")
    all_dfts = []

    for time in tqdm(range(8)):
        data, pred, dfts, tensions, biases = fit_dft2_hermite_interpolated_at_time(*args, time, **kwargs, verbose=False)
        all_dfts.append(dfts)

        data_variance += data.var()
        mae_error += abs(data - pred)
        mse_error += (data - pred) ** 2

    size = sum(el.nbytes for dfts in all_dfts for dft in dfts for el in dft) + (tensions.nbytes + biases.nbytes) * len(all_dfts)
    lines = f"""
    Original Stdev: {(data_variance / 8) ** 0.5} m/s
    Predicted MAE:  {(mae_error / 8).mean()} m/s
    Predicted RMSE: {(mse_error / 8).mean() ** 0.5} m/s

    Size/day: {size / (1000 ** 2)} MB
    Size/year: {size * 365 / (1000 ** 2)} MB
    """
    print(lines)

In [53]:
_, _, dfts = fit_dft2_at_time("MERRA2.tavg3_3d_asm_Nv.YAVG0101.nc4", "U", time=0, quantile=0.9935, verbose=False)

In [98]:
i_indices = dfts[0][2]
j_indices = dfts[0][3]
# i_indices = decode_difference_uint8(i_indices)
# i_indices = encode_rlen(i_indices)
# i_indices = encode_val_rlen(i_indices, 1)

decode_zlib(dfts[0][0])

array([154, 106, 228, ...,  65, 217, 192], dtype=uint8)

In [7]:
fit_dft2_on_day("MERRA2.tavg3_3d_asm_Nv.YAVG0101.nc4", "U", quantile=0.9935)


  0%|          | 0/8 [00:00<?, ?it/s]


    Original Stdev: 10.1023820749644 m/s
    Predicted MAE:  0.4297342300415039 m/s
    Predicted RMSE: 0.6150524051947641 m/s

    Size/day: 0.858877 MB
    Size/year: 313.490105 MB
    


In [19]:
fit_dft2_on_day("MERRA2.tavg3_3d_asm_Nv.YAVG0101.nc4", "U", quantile=0.9935)


  0%|          | 0/8 [00:00<?, ?it/s]


    Original Stdev: 10.1023820749644 m/s
    Predicted MAE:  0.4297342300415039 m/s
    Predicted RMSE: 0.6150524051947641 m/s

    Size/day: 0.849629 MB
    Size/year: 310.114585 MB
    


In [38]:
fit_dft2_hermite_interpolated_on_day("MERRA2.tavg3_3d_asm_Nv.YAVG0101.nc4", "U",
                                     quantile=0.981, skip_levels=3)


  0%|          | 0/8 [00:00<?, ?it/s]


    Original Stdev: 10.1023820749644 m/s
    Predicted MAE:  0.332945853471756 m/s
    Predicted RMSE: 0.5243341537771498 m/s

    Size/day: 0.849865 MB
    Size/year: 310.200725 MB
    


In [52]:
fit_dft2_hermite_interpolated_on_day("MERRA2.tavg3_3d_asm_Nv.YAVG0101.nc4", "U",
                                     quantile=0.98, skip_levels=3)


  0%|          | 0/8 [00:00<?, ?it/s]


    Original Stdev: 10.1023820749644 m/s
    Predicted MAE:  0.3349647521972656 m/s
    Predicted RMSE: 0.5279706831075655 m/s

    Size/day: 0.847721 MB
    Size/year: 309.418165 MB
    
